In [ ]:
!pip install beautifulsoup4
!pip install requests

In [ ]:
from datetime import datetime, timedelta

# Current date
current_date = datetime.now()

# Initialize lists to store year, month, and day for each week
years = []
months = []
days = []

# Calculate the date for each week in the past 2 years
for week in range(1, 105):
    # Subtracting weeks from the current date
    target_date = current_date - timedelta(weeks=week)

    # Extract year, month, and day
    year = target_date.year
    month = target_date.month
    day = target_date.day

    # Append to lists
    years.append(year)
    months.append(month)
    days.append(day)

# Print the first few values
print("Years:", years[:5])
print("Months:", months[:5])
print("Days:", days[:5])







Years: [2023, 2023, 2023, 2023, 2023]
Months: [9, 9, 9, 9, 9]
Days: [29, 22, 15, 8, 1]


WORKING WITHOUT CONCURRECNY!

In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

def scrape_nyt_week(week_number, end_date):
    target_date = end_date - timedelta(weeks=week_number)
    year, month, day = target_date.year, target_date.month, target_date.day

    url = f'https://www.nytimes.com/books/best-sellers/{year}/{month:02d}/{day:02d}'
    #print("url is")
    #print(url)

    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    scraped_data = []

    category_elements = soup.find_all('a', class_='css-nzgijy')
    #print("Number of categories:", len(category_elements))
    #print("category_elements:", category_elements)

    week_data = {'week': f"Week {week_number + 1}", 'data': []}  # Initialize week_data

    for i, category_element in enumerate(category_elements):
        #print("category_element:", category_element)
        category_name = category_element.text.strip()
        #print("category_name:", category_name)

        # Locate the parent section for books
        books_div = category_element.find_parent('section', class_='css-1efiz68')

        # If not found, try alternative classes
        if books_div is None:
          alternative_classes = ['css-k873nj', 'ss-1ne2zkz', 'css-1ne2zkz']
          for alt_class in alternative_classes:
            books_div = category_element.find_parent('section', class_=alt_class)
            if books_div:
              break

        if books_div:
            books = []
            for book_div in books_div.find_all('li', class_='css-1mr03gh'):
                title = book_div.find('h3', class_='css-i1z3c1').text.strip()
                #print("title", title)
                img_url = book_div.find('img')['src']
                position = int(book_div.find('meta', itemprop='position')['content'])
                description = book_div.find('p', class_='css-5yxv3r').text.strip()

                books.append({
                    'title': title,
                    'img_url': img_url,
                    'position': position,
                    'description': description
                })
                #print("books:", books)

            # Create a dictionary for the week
            category_data = {'category': category_name, 'books': books}
            week_data['data'].append(category_data)
        else:
            print(f"Warning: No corresponding div found for category {i + 1}")

    scraped_data.append(week_data)  # Append week_data to scraped_data

    return scraped_data

# End date for scraping (October 15, 2023)
end_date = datetime(2023, 10, 15)
weeks_in_two_years = 104
all_results = []
start_time = time.time()

# Iterate over the weeks
for week_number in range(weeks_in_two_years):
    result = scrape_nyt_week(week_number, end_date)
    all_results.extend(result)

# Print or use all_results as needed
end_time = time.time()
print(all_results)
elapsed_time = end_time - start_time
print(f"Scraping took {elapsed_time:.2f} seconds")

[{'week': 'Week 1', 'data': [{'category': 'Combined Print & E-Book Fiction', 'books': [{'title': 'THE RUNNING GRAVE', 'img_url': 'https://storage.googleapis.com/du-prd/books/images/9780316572101.jpg', 'position': 1, 'description': "The seventh book in the Cormoran Strike series. Strike's business partner, Robin Ellacott, goes inside a cult to rescue someone who has joined it."}, {'title': 'THE ARMOR OF LIGHT', 'img_url': 'https://storage.googleapis.com/du-prd/books/images/9780525954996.jpg', 'position': 2, 'description': 'The fifth book in the Kingsbridge series. Change and turmoil affect various aspects of society in the latter part of the 18th century.'}, {'title': 'FOURTH WING', 'img_url': 'https://storage.googleapis.com/du-prd/books/images/9781649374042.jpg', 'position': 3, 'description': 'Violet Sorrengail is urged by the commanding general, who also is her mother, to become a candidate for the elite dragon riders.'}, {'title': '12 MONTHS TO LIVE', 'img_url': 'https://storage.goog

CONCURRENT APPROACH:

In [ ]:
import aiohttp
import asyncio
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time

async def fetch(session, url):
    async with session.get(url) as response:
        return await response.text()

async def scrape_nyt_week(week_number, end_date, session):
    start_time = time.time()

    target_date = end_date - timedelta(weeks=week_number)
    year, month, day = target_date.year, target_date.month, target_date.day

    url = f'https://www.nytimes.com/books/best-sellers/{year}/{month:02d}/{day:02d}'

    async with session.get(url) as response:
        html = await response.text()

    soup = BeautifulSoup(html, 'html.parser')
    scraped_data = []

    category_elements = soup.find_all('a', class_='css-nzgijy')

    for i, category_element in enumerate(category_elements):
        category_name = category_element.text.strip()

        books_div = category_element.find_parent('section', class_='css-1efiz68')
        if books_div is None:
            books_div = category_element.find_parent('section', class_='css-k873nj')
        if books_div is None:
            books_div = category_element.find_parent('section', class_='ss-1ne2zkz')
        if books_div is None:
            books_div = category_element.find_parent('section', class_='css-1ne2zkz')

        if books_div:
            books = []
            for book_div in books_div.find_all('li', class_='css-1mr03gh'):
                title = book_div.find('h3', class_='css-i1z3c1').text.strip()
                img_url = book_div.find('img')['src']
                position = int(book_div.find('meta', itemprop='position')['content'])
                description = book_div.find('p', class_='css-5yxv3r').text.strip()

                books.append({
                    'title': title,
                    'img_url': img_url,
                    'position': position,
                    'description': description
                })

            week_data = {'category': category_name, 'books': books}
            scraped_data.append(week_data)
        else:
            print(f"Warning: No corresponding div found for category {i + 1}")

    end_time = time.time()
    #print(f"Time taken for week {week_number}: {end_time - start_time:.2f} seconds")

    return {'week': f'Week {week_number + 1}', 'data': scraped_data}

async def main():
    end_date = datetime(2023, 10, 15)
    weeks_in_two_years = 104
    all_results = []
    total_start_time = time.time()

    async with aiohttp.ClientSession() as session:
        tasks = [scrape_nyt_week(week_number, end_date, session) for week_number in range(weeks_in_two_years)]
        all_results = await asyncio.gather(*tasks)

    total_end_time = time.time()
    print(f"Total time taken for all weeks: {total_end_time - total_start_time:.2f} seconds")
    print(all_results)

# Run the asyncio event loop
await main()

Total time taken for all weeks: 15.25 seconds
[{'week': 'Week 1', 'data': [{'category': 'Combined Print & E-Book Fiction', 'books': [{'title': 'THE RUNNING GRAVE', 'img_url': 'https://storage.googleapis.com/du-prd/books/images/9780316572101.jpg', 'position': 1, 'description': "The seventh book in the Cormoran Strike series. Strike's business partner, Robin Ellacott, goes inside a cult to rescue someone who has joined it."}, {'title': 'THE ARMOR OF LIGHT', 'img_url': 'https://storage.googleapis.com/du-prd/books/images/9780525954996.jpg', 'position': 2, 'description': 'The fifth book in the Kingsbridge series. Change and turmoil affect various aspects of society in the latter part of the 18th century.'}, {'title': 'FOURTH WING', 'img_url': 'https://storage.googleapis.com/du-prd/books/images/9781649374042.jpg', 'position': 3, 'description': 'Violet Sorrengail is urged by the commanding general, who also is her mother, to become a candidate for the elite dragon riders.'}, {'title': '12 MON